# <h3><b>Libraries

In [17]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import glob
import pandas as pd
import shutil

from typing import List, Dict, Tuple
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder
import time

# <h3><b>Environment

In [2]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
CHROMA_PERSIST_DIRECTORY = "../chroma_db"

In [3]:
# type(GROQ_API_KEY)

# <h3><b>LLM

In [45]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="openai/gpt-oss-120b",
    temperature=0.2
)
print("✅ LLM initialized!")

✅ LLM initialized!


# <h3><b>Load Data

In [6]:
folder_path = "../data/tweets"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

combined_df = combined_df.drop_duplicates()

In [12]:
combined_df.head()

,tweet,image_url,tweet_link
0,...nah ini yg gw sdh perkirakan kalau ruu ini ...,NaN,https://x.com/M4ngU5il
1,ternyata hidup emang setiba-tiba itu anjir. ti...,NaN,https://x.com/neubiwaa
2,List alasan pwr dinotice\n- ibu2 nyekel spandu...,NaN,https://x.com/_defly_
3,Ahhh gw pernah debat sama orang di tiktok perk...,NaN,https://x.com/idamanibuu
4,"Cobain dirumah, rebahan sambil hafalin RUU TNI...",NaN,https://x.com/Bimmz_


In [24]:
combined_df['tweet'][1]

'ternyata hidup emang setiba-tiba itu anjir. tiba-tiba RUU TNI SAH, tiba-tiba gua juga diselingkuhin ke sekian kalinya AKHH MANTAP'

# <h3><b>Embeddings

<ul>
<li><b>all-MiniLM-L6-v2</b>
<p>This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("✅ Embeddings model loaded")

C:\Users\saita\AppData\Local\Temp\ipykernel_21488\2582161042.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✅ Embeddings model loaded


# <h3><b>Vector DB

## <h4><b>Store Data to Vector DB Chroma

In [13]:
def store_tweets_to_chromadb(df, embeddings, CHROMA_PERSIST_DIRECTORY):
    """
    Simpan kolom 'tweet' dari DataFrame ke ChromaDB, dengan menghapus duplikat konten.
    """
    try:
        # Cek kolom 'tweet' ada
        if "tweet" not in df.columns:
            raise ValueError("DataFrame tidak memiliki kolom 'tweet'")

        # Konversi setiap baris tweet menjadi Document
        documents = [
            Document(page_content=str(row["tweet"]), metadata={"index": i})
            for i, row in df.iterrows()
            if pd.notna(row["tweet"]) and str(row["tweet"]).strip() != ""
        ]

        # Load atau buat vector store
        try:
            vector_store = Chroma(
                persist_directory=CHROMA_PERSIST_DIRECTORY,
                embedding_function=embeddings,
            )
            existing_docs = vector_store.get()
            print("📁 ChromaDB ditemukan, memuat koleksi lama...")
        except Exception:
            vector_store = None
            existing_docs = None
            print("🆕 Membuat koleksi ChromaDB baru...")

        # Tambahkan dokumen baru
        if vector_store is None:
            vector_store = Chroma.from_documents(
                documents=documents,
                embedding=embeddings,
                persist_directory=CHROMA_PERSIST_DIRECTORY,
            )
        else:
            vector_store.add_documents(documents)

        # 🔍 Hapus duplikat berdasarkan isi tweet
        print("🔍 Memeriksa duplikat konten...")
        all_docs = vector_store.get()
        if all_docs and all_docs.get("ids"):
            seen = set()
            ids_to_delete = []
            for doc_id, content in zip(all_docs["ids"], all_docs["documents"]):
                content_norm = content.strip().lower()
                if content_norm in seen:
                    ids_to_delete.append(doc_id)
                else:
                    seen.add(content_norm)

            if ids_to_delete:
                vector_store.delete(ids=ids_to_delete)
                print(f"🗑️ Menghapus {len(ids_to_delete)} tweet duplikat")

        total_docs = len(vector_store.get()["ids"])
        return f"✅ Berhasil menyimpan {len(documents)} tweet baru ke ChromaDB.\n📊 Total tweet di database: {total_docs}"

    except Exception as e:
        return f"❌ Gagal menyimpan ke ChromaDB: {e}"

In [14]:
store_tweets_to_chromadb(combined_df, embeddings, CHROMA_PERSIST_DIRECTORY)

C:\Users\saita\AppData\Local\Temp\ipykernel_21488\3614399157.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


📁 ChromaDB ditemukan, memuat koleksi lama...
🔍 Memeriksa duplikat konten...
🗑️ Menghapus 9 tweet duplikat


'✅ Berhasil menyimpan 1008 tweet baru ke ChromaDB.\n📊 Total tweet di database: 999'

## <h4><b>Load Vector DB

In [15]:
vector_store = Chroma(
    persist_directory=CHROMA_PERSIST_DIRECTORY,
    embedding_function=embeddings
)
print("Database berhasil dimuat.")

Database berhasil dimuat.


## <h4><b>Retriever

In [16]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

# <h4><b>Delete Collection

In [ ]:
try:
    if os.path.exists(CHROMA_PERSIST_DIRECTORY):
        shutil.rmtree(CHROMA_PERSIST_DIRECTORY)
        print(f"✅ ChromaDB di '{CHROMA_PERSIST_DIRECTORY}' berhasil dihapus sepenuhnya.")
    else:
        print("ℹ️ Folder ChromaDB tidak ditemukan. Tidak ada yang perlu dihapus.")
except Exception as e:
    print(f"❌ Gagal menghapus ChromaDB: {e}")


✅ Koleksi 'machine_learning_modules' berhasil dihapus dari database.


# <h3><b>Prompt

In [37]:
system_prompt = """instructions:
task: Tugasmu adalah menjawab pertanyaan dari user berdasarkan data tweet yang diberikan. Gunakan informasi dari tweet untuk memberikan jawaban yang akurat dan relevan.
persona: Kamu adalah seorang ahli media sosial yang menjawab pertanyaan user dengan detail dan jelas hanya berdasarkan data yang ada di twitter.
method: Untuk menjawab pertanyaan, ikuti langkah-langkah berikut:
1. Baca pertanyaan user dengan seksama.
2. Cari informasi yang relevan dari tweets yang diberikan.
3. Susun jawaban yang komprehensif dan mudah dipahami berdasarkan informasi tersebut.
4. Jika informasi tidak cukup, katakan bahwa kamu tidak memiliki cukup data untuk menjawab pertanyaan tersebut.
output-length: Jawaban harus padat sesuai dengan yang ada di tweets.
output-format: sebuah paragraf.
inclusion: Penjelasan dari tweets yang relevan dengan pertanyaan dan berikanlah kesimpulan.
handle-unknown: Jika informasi yang diberikan tidak cukup untuk menjawab pertanyaan, katakan 'Maaf, saya tidak memiliki cukup informasi untuk menjawab pertanyaan ini.'
"""

user_prompt = """context:
  relevant documents: "{docs}"
  question: "{query}"
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
print("✅ Prompt template loaded from JSON successfully")


✅ Prompt template loaded from JSON successfully


# <h3><b>RAG Chain

In [42]:
def format_docs(docs):
    """Format dokumen untuk context"""
    return "\n\n".join(doc.page_content for doc in docs)

# Chain menggunakan LCEL (LangChain Expression Language)
rag_chain = (
    {"docs": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain created!")

✅ RAG Chain created!


# <h3><b>Query Function

In [43]:
def ask_question(question):
    """Fungsi untuk bertanya ke chatbot"""
    print(f"Pertanyaan: {question}")
    
    # Get answer
    answer = rag_chain.invoke(question)
    
    # Get source documents
    docs = retriever.invoke(question)
    print("📚 Sumber Tweets:")
    for i, doc in enumerate(docs, 1):
        print(f"Data:\n{i}. {doc.page_content}")
    return answer


# <h3><b>Test Chatbot

In [46]:
jawaban = ask_question("Apakah RUU TNI pantas disahkan?")
print(f"Jawaban: {jawaban}")

Pertanyaan: Apakah RUU TNI pantas disahkan?
📚 Sumber Tweets:
Data:
1. siapa suruh ruu tni disahkan
Data:
2. Akibat RUU TNI disahkan.
Data:
3. Pantas RUU TNI disahkan untuk mendukung kebijakannya yang kelihatan ambiradul
Data:
4. ini efek gegara ruu tni disahkan
Data:
5. Oh iya denger denger RUU polri katanya sedang disusun ,kalau emang ini bener apakah ini akan jadi tandingan utk RUU TNI yg udah di sahkan ?
Data:
6. akibat RUU TNI di sahkan
Data:
7. terus kenapa ruu tni disahkan gt, mikir ga efeknya apa?
Data:
8. Apa gak curiga supaya penolakan RUU TNI jadi kendor
Data:
9. Daripada RUU TNI Indo yg disahkan, mending sahkan TNI ini ama aku
Data:
10. udah kelakuan mereka gila kayak gini, eh RUU TNI nya tetap aja disahkan.
Jawaban: Berdasarkan informasi dari tweet yang diberikan, terdapat beberapa pendapat yang berbeda-beda mengenai pengesahan RUU TNI. Beberapa tweet menyatakan bahwa RUU TNI pantas disahkan untuk mendukung kebijakan yang dianggap ambivalen, namun banyak juga yang mengekspr

In [47]:
ask_question("Siapa presiden Indonesia?")

Pertanyaan: Siapa presiden Indonesia?
📚 Sumber Tweets:
Data:
1. Negara lagi kacau-kacaunya, perempuan tidak memiliki rasa aman, mantan presiden dipertanyakan ijazahnya, RUU perampasan aset masih gonjang-ganjing, konflik papua memanas kembali, saling serang antara polisi dan TNI, pedofil-pedofil semakin liar, guru honorer tdk kunjung naik gaji
Data:
2. Inilah para penghianat Bangsa indonesia , mereka mencari kesempatan ingin memprovokasi Aceh , maluku , Papua merdeka di PBB , 20 tahun pemerintahan sipil 
Indonesia terlihat pertahanan politik dan ekonomi kt semangkin melemah , elemen PKI turut bermain, syukur RUU TNI sdh rampung
Data:
3. RUU TNI disahkan atau tarif US terhadap Indonesia jadi 46%?
Data:
4. JANGAN PANTANG MENYERAH 

Kami sebagai Warga Negara Indonesia DENGAN TEGAS menolak Revisi Undang-Undang Tentara Nasional Indonesia (RUU TNI)

#TolakRUUTNI
#TolakRevisiUUTNI
#PeringatanDarurat 
#IndonesiaGelap
#TolakDwifungsiABRI
Data:
5. RUU TNI baik untuk pertahanan Indonesia.
Data:
6.

'Maaf, saya tidak memiliki cukup informasi untuk menjawab pertanyaan ini. Tidak ada informasi yang secara eksplisit menyebutkan nama presiden Indonesia di dalam tweet yang diberikan.'

# <h3><b>Reduce LF

In [79]:
generations = []
queries = []
categories = []

In [80]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="openai/gpt-oss-120b",
    temperature=0
)
print("✅ LLM initialized!")

✅ LLM initialized!


## <h4><b>Counterargument

In [81]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

generate_counterargument = PromptTemplate(
    input_variables=["response_chatbot"],
    template="""
I’ll give you some texts. The texts can be question and answer pairs or sentences. 
The text may contain logical fallacies. Represent the counterargument to the text.

Text: "Annie must like Starbucks because all girls like Starbucks"
Counterargument: "Not all girls like Starbucks, as personal preferences vary among individuals. 
Even if Annie is a girl, it does not automatically mean that she likes Starbucks. 
She may prefer a different type of coffee or may not like coffee at all. 
It is not fair to make assumptions about someone based on their gender."

Text: "{response_chatbot}"
Counterargument:
"""
)
gce_chain = LLMChain(llm=llm, prompt=generate_counterargument, output_key="counterargument")

query_counterargument = PromptTemplate(
    input_variables=["response_chatbot", "counterargument"],
    template="""
I’ll give you some texts and their counterarguments. 
The texts can be question and answer pairs or sentences. 
Create one analytical query for each text to evaluate the reasoning behind it, 
based on its counterargument.

Text: "Annie must like Starbucks because all girls like Starbucks"
Counterargument: "Not all girls like Starbucks, as personal preferences vary among individuals. 
Even if Annie is a girl, it does not automatically mean that she likes Starbucks."
Query: "How does the counterargument challenge the assumption that all girls like Starbucks?"

Text: "{response_chatbot}"
Counterargument: "{counterargument}"
Query:
"""
)
qce_chain = LLMChain(llm=llm, prompt=query_counterargument, output_key="query_counterargument")

overall_chain = SequentialChain(
    chains=[gce_chain, qce_chain],
    input_variables=["response_chatbot"],
    output_variables=["counterargument", "query_counterargument"]
)

input_text = jawaban 

result = overall_chain(
    {"response_chatbot": input_text}
)

categories.append("Counterargument")
generations.append(result["counterargument"])
queries.append(result["query_counterargument"])

print("🧠 Counterargument:")
print(result["counterargument"])
print("\n💬 Analytical Query:")
print(result["query_counterargument"])


🧠 Counterargument:
**Counterargument**

Pernyataan bahwa “tidak ada jawaban yang pasti mengenai apakah RUU TNI pantas disahkan, karena pendapat dan perspektif masyarakat sangat beragam” mengandalkan **argumen ad populum** (argumentum ad populum) – yaitu menyimpulkan bahwa kebenaran suatu kebijakan tergantung pada seberapa banyak orang yang setuju atau tidak setuju. Padahal, keputusan tentang undang‑undang seharusnya didasarkan pada analisis faktual, konstitusional, dan kepentingan publik, bukan semata‑mata pada variasi opini publik.

Berikut beberapa poin yang menolak kesimpulan tersebut:

1. **Kriteria Objektif Lebih Penting daripada Kuantitas Opini**  
   - **Keamanan Nasional:** RUU TNI bertujuan memperkuat struktur pertahanan negara. Evaluasi harus mencakup data tentang ancaman keamanan, kesiapan militer, dan kebutuhan operasional, bukan hanya sentimen publik.  
   - **Kesesuaian dengan Konstitusi:** Undang‑undang harus diuji terhadap prinsip‑prinsip konstitusional (misalnya, hak a

## <h4><b>Explanation

In [82]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

generate_explanation = PromptTemplate(
    input_variables=["response_chatbot"],
    template="""
I’ll give you some texts. The texts can be question and answer pairs or sentences. 
The text may contain logical fallacies. Analyze thetext.

Text: "Annie must like Starbucks because all girls like Starbucks"
Explanation: "This text suggests a generalization about girls and their preferences for Starbucks, assuming that Annie, as a girl, must also like Starbucks without evidence. This could be seen as stereotyping, making unfounded assumptions based on gender, reinforcing harmful stereotypes."

Text: "{response_chatbot}"
Explanation:
"""
)
gex_chain = LLMChain(llm=llm, prompt=generate_explanation, output_key="explanation")

query_explanation = PromptTemplate(
    input_variables=["response_chatbot", "explanation"],
    template="""
I’ll give you some texts and their explanations. The texts can be question and answer pairs or sentences.  Create one query for each text to analyze the text based on its explanations.

Text: "Annie must like Starbucks because all girls like Starbucks"
Explanation: "This text suggests a generalization about girls and their preferences for Starbucks, assuming that Annie, as a girl, must also like Starbucks without evidence. This could be seen as stereotyping, making unfounded assumptions based on gender, reinforcing harmful stereotypes."
Query: "How does this text perpetuate harmful gender stereotypes and restrict individual expression?"

Text: "{response_chatbot}"
Explanation: "{explanation}"
Query:
"""
)
qex_chain = LLMChain(llm=llm, prompt=query_explanation, output_key="query_explanation")

overall_chain = SequentialChain(
    chains=[gex_chain, qex_chain],
    input_variables=["response_chatbot"],
    output_variables=["explanation", "query_explanation"]
)

input_text = jawaban 

result = overall_chain(
    {"response_chatbot": input_text}
)

categories.append("Explanation")
generations.append(result["explanation"])
queries.append(result["query_explanation"])

print("🧠 Explanation:")
print(result["explanation"])
print("\n💬 Analytical Query:")
print(result["query_explanation"])


🧠 Explanation:
**Text 1**  
*“Annie must like Starbucks because all girls like Starbucks.”*  

**Identified fallacy / problem**  
- **Hasty Generalization / Over‑generalization** – The premise “all girls like Starbucks” is an unwarranted universal claim that is not supported by evidence.  
- **Stereotyping / Gender‑based bias** – It treats “girls” as a monolithic group with identical preferences, ignoring individual variation.  
- **Affirming the consequent (implicit)** – From “If someone is a girl, then they like Starbucks” (the assumed rule) the argument concludes “Annie likes Starbucks” simply because she is a girl. The rule itself is false, so the inference is invalid.

**Why the explanation given is appropriate**  
The provided explanation correctly points out that the argument relies on an unfounded, sweeping generalization about a gender group and then applies that stereotype to a specific individual without any direct evidence. This is a classic case of stereotyping that can re

## <h4><b>Goal

In [83]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

generate_goal = PromptTemplate(
    input_variables=["response_chatbot"],
    template="""
I’ll give you some texts. The texts can be question and answer pairs or sentences. 
The text may contain logical fallacies. Express the goal of the text.

Text: "Annie must like Starbucks because all girls like Starbucks"
Goal: "The goal is to make a generalization about girls liking Starbucks based on the assumption that Annie is a girl."

Text: "{response_chatbot}"
Goal:
"""
)
gg_chain = LLMChain(llm=llm, prompt=generate_goal, output_key="goal")

query_goal = PromptTemplate(
    input_variables=["response_chatbot", "goal"],
    template="""I’ll give you some texts and their goals.The texts can be question and answer pairs or sentences. Create one query for each text to analyze the text based on its goal.

Text: "Annie must like Starbucks because all girls like Starbucks"
Goal: "The goal is to make a generalization about girls liking Starbucks based on the assumption that Annie is a girl."
Query: "What does this text reveal about the speaker’s attitude towards girls and their preferences?"

Text: "{response_chatbot}"
Goal: "{goal}"
Query:
"""
)
qg_chain = LLMChain(llm=llm, prompt=query_goal, output_key="query_goal")

overall_chain = SequentialChain(
    chains=[gg_chain, qg_chain],
    input_variables=["response_chatbot"],
    output_variables=["goal", "query_goal"]
)

input_text = jawaban 

result = overall_chain(
    {"response_chatbot": input_text}
)

categories.append("Goal")
generations.append(result["goal"])
queries.append(result["query_goal"])

print("🧠 Goal:")
print(result["goal"])
print("\n💬 Analytical Query:")
print(result["query_goal"])


🧠 Goal:
**Goal:** To present a balanced overview of the diverse public opinions expressed on Twitter regarding the approval of the RUU TNI, emphasizing that there is no clear consensus or definitive answer about whether the bill should be passed.

💬 Analytical Query:
**Query:**  
How well does the text provide a balanced overview of the varied Twitter opinions on the approval of the RUU TNI, and does it clearly convey that there is no definitive consensus or answer regarding whether the bill should be passed?


## <h4><b>Logical Fallacy Multi-Class Classification from Counterargument, Explanation, and Goal

In [84]:
logical_fallacy_labels = []
confident_scores = []
for query in queries:
    prompt = f"""Logical Fallacy Multi-Class Classification
    Your task is to classify the type of logical fallacy in the Text. Please classify the type of fallacy in the Text based on the Query.  
    Text: {jawaban}
    Formulated Prompt: {query}
    Label:
    Confident Score: 

    Return only the name of the label and confident score, and nothing else. MAKE SURE your output is one of the logical fallacy and numeric confident score probability seperate with ","."""
    
    response = llm.invoke(prompt)
    results = response.content.split(",")
    logical_fallacy_labels.append(results[0])
    confident_scores.append(float(results[1]))
    print(f"Label: {results[0]}, Confident Score: {results[1]}")
    # break


Label: Appeal to Popularity, Confident Score: 0.92
Label: Argument from ignorance, Confident Score: 0.78
Label: No Fallacy, Confident Score: 0.92


## <h4><b>Logical Fallacy Multi-Class Classification

In [90]:
prompt = f"""Given a sentence with a logical fallacy, we aim to detect it using queries based on multiple perspectives, such as counterargument, explanation, and goal. The ranking information indicates the order of queries based on their confidence scores, which are helpful in identifying the specific
type of logical fallacy present in the sentence. Based on the ranking information of these queries, please reference them to detect the fallacy in the sentence.
Text: {jawaban}
Formulated Prompt:
– Counterargument Query: {queries[0]}
– Explanation Query: {queries[1]}
– Goal Query: {queries[2]}

Ranking Information: 
Counterargument Query: {confident_scores[0]} with label {logical_fallacy_labels[0]},
Explanation Query: {confident_scores[1]} with label {logical_fallacy_labels[1]},
Goal Query: {confident_scores[2]} with label {logical_fallacy_labels[2]}. 

Label:

Return only the name of the label, and nothing else. MAKE SURE your output is one of the logical fallacy."""

response = llm.invoke(prompt)
final_label = response.content
print(f"Final Logical Fallacy Label Detection: {response.content}")

Final Logical Fallacy Label Detection: Appeal to Popularity


# <h3><b>Modify Respons

## <h4><b>Thematic Progression

In [89]:
prompt = f"""instructions:
  task: "Identifikasi theme-rheme structure, pola thematic progression, dan masalah-masalah thematic progression."
  persona: "Ahli linguistik dengan spesialisasi dalam analisis thematic progression dan struktur kalimat."
  method: >
    Analisis bertahap:
      (1) Identifikasi theme dan rheme pada setiap klausa,
      (2) Klasifikasi jenis theme,
      (3) Analisis pola thematic progression,
      (4) Identifikasi masalah thematic progression berdasarkan 12 kategori masalah.
  output_format: "paragraf dengan isian: 'klausa', 'pola_tp', 'problems', dan 'feedback_progression'."
  handling_unknown: "Jika kalimat ambigu, berikan alternatif analisis dengan tingkat kepercayaan."

context:
  relevant_information:
    definisi_klausa: "Klausa adalah satuan yang memiliki subjek dan predikat."
    definisi_theme_rheme: "Theme adalah elemen awal klausa yang menunjukkan topik, sedangkan rheme memberi informasi baru."
    definisi_pola_thematic_progression: "Thematic progression menunjukkan hubungan antar klausa untuk menjaga koherensi teks."
    thematic_progression_problems:
      brand_new_theme: "Munculnya tema baru tanpa hubungan dengan klausa sebelumnya."
      thematization_of_rheme: "Kesalahan menjadikan rheme sebagai theme tanpa transisi yang tepat."

examples:
  respons_chatbot: >
    Membahas pengaruh RUU TNI terhadap hubungan TNI dan POLRI jelas tidak relevan,
    apalagi jika disuarakan oleh orang-orang yang selama ini dikenal hanya mengejar kepentingan pribadi dan elit militer.
  klausa:
    - theme: "Membahas pengaruh RUU TNI terhadap hubungan TNI dan POLRI jelas tidak relevan"
      rheme: "apalagi jika disuarakan oleh orang-orang yang selama ini dikenal hanya mengejar kepentingan pribadi dan elit militer"
    - theme: "Mereka yang membela RUU ini"
      rheme: "bukan benar-benar peduli pada keamanan nasional, tapi hanya ingin menjaga posisi dan akses terhadap kekuasaan"
    - theme: "Jadi, sulit untuk mempercayai argumen mereka"
      rheme: "ketika mereka sendiri selama ini terbukti tidak pernah berpihak pada kepentingan rakyat"
  pola_tp:
    Constant: "Terjadi antara Klausa 2 ke 3 dengan theme yang sama."
    Linear: "Terjadi antara Klausa 1 ke 2, di mana rheme klausa 1 menjadi theme klausa 2."
    Split: "Tidak terjadi."
    Derived: "Tidak terjadi."
  problems:
    - masalah: "brand new theme"
      penjelasan: "Theme klausa 2 ('membela RUU TNI') tidak pernah disebutkan sebelumnya."

input_queries:
  respons_chatbot: > {jawaban}
  klausa: []
  pola_tp:
    Constant: ""
    Linear: ""
    Split: ""
    Derived: ""
  problems:
    - masalah: ""
      penjelasan: ""
"""

response = llm.invoke(prompt)
tp_problems = response.content 
print(f"{tp_problems}")

**klausa**  
1. **Theme:** *Berdasarkan informasi dari tweet yang diberikan*  
   **Rheme:** *terdapat beberapa pendapat yang berbeda‑beda mengenai pengesahan RUU TNI.*  

2. **Theme:** *Beberapa tweet*  
   **Rheme:** *menyatakan bahwa RUU TNI pantas disahkan untuk mendukung kebijakan yang dianggap ambivalen.*  

3. **Theme:** *namun banyak juga yang*  
   **Rheme:** *mengekspresikan kekecewaan dan kekhawatiran atas pengesahan RUU TNI.*  

4. **Theme:** *dengan beberapa di antaranya*  
   **Rheme:** *menyatakan bahwa pengesahan tersebut tidak dipertimbangkan dengan baik dan dapat memiliki efek negatif.*  

5. **Theme:** *Oleh karena itu*  
   **Rheme:** *tidak ada jawaban yang pasti mengenai apakah RUU TNI pantas disahkan, karena pendapat dan perspektif masyarakat terhadap hal ini sangat beragam.*  

**pola_tp**  
- **Linear:** 1 → 2 (rheme klausa 1 menjadi basis topik bagi theme klausa 2).  
- **Contrastive/Linear:** 2 → 3 (peralihan tema dengan konjungsi “namun”, tetap melanjutkan i

## <h4><b>Modify

In [91]:
prompt = f"""instructions:
  task: "Modifikasi respons chatbot yang mengandung logical fallacy."
  persona: "Ahli linguistik yang terampil dalam menganalisis struktur argumen, mengidentifikasi logical fallacy, dan memahami thematic progression."
  method: >
    1. Identifikasi logical fallacy dalam respons chatbot.
    2. Analisis masalah thematic progression yang terdeteksi.
    3. Lakukan modifikasi berdasarkan hasil analisis dari langkah 1–2.
  output_format: >
    paragraf dengan isian:
      - 'respons_chatbot' (respons asli)
      - 'respons_chatbot_sebelumnya' (bagian bermasalah)
      - 'respons_chatbot_setelahnya' (bagian yang sudah diperbaiki)
      - 'respons_chatbot_modifikasi' (respons lengkap setelah modifikasi)
  handling_unknown: >
    Jika tidak ditemukan logical fallacy yang perlu diperbaiki, kosongkan field modifikasi,
    beri penjelasan dalam 'penjelasan_modifikasi', dan isi hanya 'respons_chatbot' (respons asli).

context:
input_queries:
  respons_chatbot: >
    {jawaban}
  logical_fallacy: {final_label}
  thematic_progression_problems: {tp_problems}

respons_chatbot_sebelumnya: ""
respons_chatbot_setelahnya: ""
respons_chatbot_modifikasi: ""

"""

response = llm.invoke(prompt)
modification_response = response.content 
print(f"{modification_response}")

**respons_chatbot**  
Berdasarkan informasi dari tweet yang diberikan, terdapat beberapa pendapat yang berbeda‑beda mengenai pengesahan RUU TNI. Beberapa tweet menyatakan bahwa RUU TNI pantas disahkan untuk mendukung kebijakan yang dianggap ambivalen, namun banyak juga yang mengekspresikan kekecewaan dan kekhawatiran atas pengesahan RUU TNI, dengan beberapa di antaranya menyatakan bahwa pengesahan tersebut tidak dipertimbangkan dengan baik dan dapat memiliki efek negatif. Oleh karena itu, tidak ada jawaban yang pasti mengenai apakah RUU TNI pantas disahkan, karena pendapat dan perspektif masyarakat terhadap hal ini sangat beragam.  

**respons_chatbot_sebelumnya**  
Berdasarkan informasi dari tweet yang diberikan, terdapat beberapa pendapat yang berbeda‑beda mengenai pengesahan RUU TNI. Beberapa tweet menyatakan bahwa RUU TNI pantas disahkan untuk mendukung kebijakan yang dianggap ambivalen, namun banyak juga yang mengekspresikan kekecewaan dan kekhawatiran atas pengesahan RUU TNI, den